# Chapter 3: Provision, Run and Monitor a Federated Project

Welcome to Chapter 3 of the course 5 minutes to Federated Learning with NVIDIA FLARE!

In chapter 2, we learned how to develop a federated application using NVIDIA FLARE's core APIs, as well as how to configure it and run it in a simulated environment. In this chapter, we take it to the next level by introducing project provisioning, PoC mode and project monitoring in NVIDIA FLARE. We first introduce the concept of project provisioning and explain how FLARE generates a federated project for real-world scenario. Then we introduce the Proof-of-Concept (PoC) mode in NVIDIA FLARE, a suite of command-line tools that allow FL developers to run a provisioned project in a sandbox environment on a single PC. Next, we walk you through NVIDIA FLARE's tools and APIs that allow you to monitor and track the progress of the federated project. Finally, we illustrate all these features with a hands-on example of CIFAR10 image classification using Pytorch.

After this chapter, you will:
- Have a basic understanding of project provision, PoC mode and project monitoring in NVIDIA FLARE
- Be able to generate a federated project that can be deployed as a real-world application 
- Be able to run a provisioned federated job in a sanbox environment 
- Be able to track and monitor a running federated project using FLARE's tools and APIs


# Provisioning in NVIDIA FLARE

In Chapter 1, we saw togther how to develop an example federated application using NVIDIA FLARE's APIs, and run the application in a simulated environment using FL Simulator. In that example, we defined 1 server and 2 clients as participants of the FL project. The generation and management of the participants are all managed by the FL Simulator tool, and we didn't really have the need to set them up. While this is all convenient for prototyping, for real-world federated applications, we need more fine-grained control in setting up different participants. 

In real-world FL deployment, different participants have their own clearly defined roles. Apart from server and clients, FL projects are usually assigned with administrators who manage and monitor project status. Proper user authentication and authorization mechanisms need to be set up to ensure a secure deployment environment for all participants to establish mutually-trusted communication channels. Also, due to the distributed nature of FL projects, all participants would need to be able to join the FL system from different locations through secure network connections, or in a dynamic way. On top of all these, extra security and resource management features might also be required due to different participants requirements in terms of privacy and compute resources.

To have a managed way to properly set up an FL project and account for all the aspects mentioned above, NVIDIA FLARE introduces the concept of [**Provisioning**](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#provisioning-in-nvidia-flare). In FLARE, **provisioning is the process of settting up a federated project and participants for a scalable, extensible and secure real world deployment**. Provisioning in NVIDIA FLARE is used to:
- Establish the identities of the server, clients, and administrators in the federation
- Generate startup kit for each participant
- Set up secure communication channels between participants

<img src="../images/provision.png" alt="NVFLARE Provision" width=30% />

In FLARE, provisioning can be done using either the CLI tool `nvflare provision` or webUI-based [Dashboard](https://nvflare.readthedocs.io/en/main/user_guide/dashboard_ui.html). In this chapter, we only focus on the CLI tool and we will cover Dashboard in the next chapter. 

The provisioning process typically involves the following steps:
1. Configure the project using a configuration file
2. Use the FLARE provisioning tool to generate startup kits from project configuration file
3. Distribute the startup kits to participants

Let's walk through these steps together.

### 1. Configure the project

The input to the povisioning process is a project configuration file (for instance typically `project.yaml`). This configuration is used to set up the FL project, and generally includes definitions for:
- Project meta-data
- Participants of the project
- Builders to build project workspace

An example project configuration file is provided in [`../files/project.yml`](../files/project.yml). This file was generated using the `nvflare provision` CLI tool without any arguments. You can also find master templates for more complete project configurations [here](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#default-project-yml-file).

> **NOTE**:
> When running this CLI tool without arguments, you will be prompted to select whether the generated configuration file should include [*high-availability*](https://nvflare.readthedocs.io/en/main/programming_guide/high_availability.html) features. We will not cover *high-availability* in this course, but the example file was generated without high-availability features. 

Let us look at the content of [`../files/project.yaml`](../files/project.yml) together (some of the details and comments are removed to faciliate explanation):
```yaml
api_version: 3
name: example_project
description: NVIDIA FLARE sample project yaml file

participants:
  - name: server1
    type: server
    org: nvidia
    fed_learn_port: 8002
    admin_port: 8003

  - name: site-1
    type: client
    org: nvidia

  - name: site-2
    type: client
    org: nvidia
    
  - name: admin@nvidia.com
    type: admin
    org: nvidia
    role: project_admin

builders:
  - path: nvflare.lighter.impl.workspace.WorkspaceBuilder
      ...
  - path: nvflare.lighter.impl.template.TemplateBuilder
      ...
  - path: nvflare.lighter.impl.static_file.StaticFileBuilder
      ...
  - path: nvflare.lighter.impl.cert.CertBuilder
      ...
  - path: nvflare.lighter.impl.signature.SignatureBuilder
```

The configuration file is organized into the following 3 sections:

**Project mete-data**
  - The `api_version`: for current release of NVIDIA FLARE, the `api_version` is set to 3
  - The `name` of the FL project, in this case, `example_project`
  - And a short `description` of the FL project

**Participants**

This section is a crucial part that defines all the [participants](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#participant) involved in the project. There are multiple types of participants defined in FLARE, most common types include `server`, `client` and `admin`. Some participants are referred to as [Sites](https://nvflare.readthedocs.io/en/main/user_guide/security/terminologies_and_roles.html#site), which represent computing systems that run FLARE applications, for instance, the `server` and `client`. While some participants are referred to as [Users](https://nvflare.readthedocs.io/en/main/user_guide/security/terminologies_and_roles.html#user), who are human participants, with different access priviledges to query, monitor or manage the project, for instance, the `admin`. Developers can aslo include other types for instance the `overseer` for high-availability mode, or even add custom types, but we will not go into details on this in this course. 

As we can see in the example `project.yml` file, the following participants are defined:
- `server`: here we defined 1 server. The name of the server should in general be a [fully qualified domain name](https://en.wikipedia.org/wiki/Fully_qualified_domain_name) (FQDN) to make sure that other participants can establish network connections to the server from any location. The server name can also be a system-wide known hostname.
- `client`s: here we defined 2 clients with names `site-1` and `site-2`.
- `admin`: here we defined 1 `project_admin` with name `admin@nvidia.com`, who has the most elevated access priviledges in the whole project. There can only 1 project admin for each project.

**Builders**

In NVIDIA FLARE provisioning, [builders](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#builder) are a series of Python classes that work together to generate startup kits for all participants, based on the configurations defined in the project configuration file. Builders create various aspects of the startup kits for all participants, such as workspace structure, configuration files, and security credentials. Builders specified in a configuration file will be invoked in the same order as listed in the `builders` section. This example `project.yml` file shows the usage of common builders:
- `WorkspaceBuilder`: Creates the basic workspace structure
- `TemplateBuilder`: Processes template files
- `StaticFileBuilder`: Copies static files into the workspace
- `CertBuilder`: Generates certificates and keys for secure communication
- `SignatureBuilder`: Creates signatures for tamper-proofing

Developers can also create custom builders to construct more specific and customized provisioning output. We will not deep dive into how builders work and how they can be customized. For now, the most important thing to know about builders is that they execute in specific order to build the startup kits as results of a provisioning process.


### 2. Generate startup kits

Let's run the provisioning process with the example configuration file [`../files/project.yml`/](../files/project.yml/):

In [1]:
!rm -rf ./temp-workspace
!nvflare provision -p ../files/project.yml -w ./temp-workspace

Project yaml file: /flare/notebooks/../files/project.yml.
Generated results can be found under /flare/notebooks/./temp-workspace/example_project/prod_00. 


Now let's check the structure of the output generated by provisioning:

In [2]:
!tree ./temp-workspace -L 4

./temp-workspace
└── example_project
    ├── prod_00
    │   ├── admin@nvidia.com
    │   │   ├── local
    │   │   ├── startup
    │   │   └── transfer
    │   ├── server1
    │   │   ├── local
    │   │   ├── readme.txt
    │   │   ├── startup
    │   │   └── transfer
    │   ├── site-1
    │   │   ├── local
    │   │   ├── readme.txt
    │   │   ├── startup
    │   │   └── transfer
    │   └── site-2
    │       ├── local
    │       ├── readme.txt
    │       ├── startup
    │       └── transfer
    ├── resources
    │   ├── aws_template.yml
    │   ├── azure_template.yml
    │   └── master_template.yml
    └── state
        └── cert.json

21 directories, 7 files


We can see that provisioning generates a well-defined directory hierarchy.

At the top level, we have a root folder with the FL project's name, in this case, `example_project`. Under `example_project`, we can see multiple subfolders:
- `resources`: this directory may contain shared resources or additional files needed for the project.
- `state`: this is a directory to maintain state information about the provisioning process or current status of participants.
- `prod_NN`: this folder contains the **startup kits** generated from a successful provisioning command for all participants. The number `NN` increases with each successful provision run, indicating different provisioning sessions. In this example case, since we are running the provisioning command for the first time, the folder name is `prod_00`.

**A startup kit is a folder with a set of files, scripts and credentials generated by provisioning for each participant in the FL project**. A startup kit is local to a specific participant and typically include:
- Authentication credentials
- Authorization policies
- Signatures for tamper-proof mechanisms
- Convenient shell scripts for launching the participant

We can see that in this example, there are 4 startup kits generated, one for each participants, i.e., 1 server, 2 clients and 1 admin. Each folder is named after its corresponding participant's name as indicated in the configuration file. 

Let's look into the content of the generated startup kits for the server, clients and admin. 

Server and clients have similar startup kit content structure, so let's just display the server startup kit files with the following command:


In [3]:
!tree ./temp-workspace/example_project/prod_00/server1/

./temp-workspace/example_project/prod_00/server1/
├── local
│   ├── authorization.json.default
│   ├── log.config.default
│   ├── privacy.json.sample
│   └── resources.json.default
├── readme.txt
├── startup
│   ├── fed_server.json
│   ├── rootCA.pem
│   ├── server.crt
│   ├── server.key
│   ├── signature.json
│   ├── start.sh
│   ├── stop_fl.sh
│   └── sub_start.sh
└── transfer

4 directories, 13 files


Different subfolders are organized as follows: 
- The `local` subfolder: this folder contains configuration files for site-specific local policies for authorization, logging, privacy and resource access. Each site can modify these configuration files to set up their local policies. We will see more details on local site policy management in the next chapter.
- The `startup` subfolder: this folder contains shell script for a participant to start / join (`start.sh`) or leave the FL project (`stop_fl.sh`). It also contains certificates, signature and key files, which are essential to maintain secure connections between different participants. Noted that the signatures and certificates files are integral to ensuring the security and authenticity of a participants. Manual modification of these files after provisioning may prevent a participant to connect to the FL system. 
- The `transfer` subfolder is used to store artifacts such as custom application, scripts or files during project runtime.

Now let's look at the content of `admin`'s startup kit:

In [4]:
!tree ./temp-workspace/example_project/prod_00/admin@nvidia.com/

./temp-workspace/example_project/prod_00/admin@nvidia.com/
├── local
├── startup
│   ├── client.crt
│   ├── client.key
│   ├── fed_admin.json
│   ├── fl_admin.sh
│   ├── readme.txt
│   └── rootCA.pem
└── transfer

4 directories, 6 files


The `admin`'s startup kit has similar structure. 
- The `local` subfolder is empty, since local policies are not needed for `admin`, a human participant who manages the project.
- In the `startup` subfolder, apart from certificates and keys, the `fl_admin.sh` shell script allows the `admin` user to login to [**FLARE Console**](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html#operating-nvflare) to perform management tasks from anywhere with secure network connection. We will look at the **FLARE Console** later.
- The `transfer` folder in `admin`'s startup kit is the default location to hold various runtime data, for instance, downloads of federated jobs from the server workspace when they finish. It can also be used to link or hold federated applications developed using FLARE, so that the `admin` user can easily submit the application to the server for running. We will see how this works in details later.

### 3. Distribute the startup kits to participants

Now with startup kits generated, the last step is to distribute these kits to the corresponding participants. You can use email, sftp etc. to do so, as long as you can ensure that it is secure. In general, each site should have an organization admin to receive or download the startup kits. The organization admin can then install their own packages, start the services, map the data location, and instrument the authorization policies and organization level site privacy policies.

**That's it! We have learned how to properly set up a federated project and generate startup kits for all participants using FLARE's provisioning process!**

# Proof-of-Concept Mode

Now let's put what we've learn about FLARE provisioning into action: let us run a federated application with properly provisioned server, clients and admin!

While it sounds exciting, this is however not a trivial task. We would have to set up distributed computing environments for the server, clients and admin. It would be nice if there is a way to alleviate this set-up. Introducing NVIDIA FLARE's Proof-of-Concept, or [PoC mode](https://nvflare.readthedocs.io/en/main/user_guide/poc_command.html). PoC mode allows users to test the features of FLARE provisioning and deployment on a single machine, without the overhead of a true distributed deployment and the need to establish secure communication between server and client systems.

Compared to the FL Simulator, where the job run is automated on a single system, PoC mode allows you to establish and connect distinct server, client and admin in different processes. PoC mode also opens possibility for admin to orchestrate and manage the project using the [FLARE Console](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html#operating-nvflare) or [FLARE API](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html), making it a useful tool in preparation for a real-world distributed deployment.

Developers often start with PoC mode to test their applications locally before transitioning to a production environment where provisioning is essential. The transition involves moving from a simplified setup to a more secure configuration that includes mutual authentication and authorization policies.

To get started, let's look at the NVFlare CLI usage for PoC mode:

In [5]:
!nvflare poc -h

usage: nvflare poc [-h] [--prepare] [--start] [--stop] [--clean]
                   {prepare,prepare-jobs-dir,start,stop,clean} ...

options:
  -h, --help            show this help message and exit
  --prepare             deprecated, suggest use 'nvflare poc prepare'
  --start               deprecated, suggest use 'nvflare poc start'
  --stop                deprecated, suggest use 'nvflare poc stop'
  --clean               deprecated, suggest use 'nvflare poc clean'

poc:
  {prepare,prepare-jobs-dir,start,stop,clean}
                        poc subcommand
    prepare             prepare poc environment by provisioning local project
    prepare-jobs-dir    prepare jobs directory
    start               start services in poc mode
    stop                stop services in poc mode
    clean               clean up poc workspace


As we can see, the PoC mode is composed of multiple subcommands. Let's quickly walk through each of the subcommands.

#### `nvflare poc prepare`

This is the first command to run when launching a PoC mode project. This command generates a local project workspace with startup kits for all participants. Internally it runs the provision process either with a user-specified configuration file or with a default one.

Here is the help info for `nvflare poc prepare`:

In [6]:
!nvflare poc prepare -h

usage: nvflare poc prepare [-h] [-n [NUMBER_OF_CLIENTS]] [-c [CLIENTS ...]]
                           [-he] [-i [PROJECT_INPUT]] [-d [DOCKER_IMAGE]]
                           [-debug]

options:
  -h, --help            show this help message and exit
  -n [NUMBER_OF_CLIENTS], --number_of_clients [NUMBER_OF_CLIENTS]
                        number of sites or clients, default to 2
  -c [CLIENTS ...], --clients [CLIENTS ...]
                        Space separated client names. If specified,
                        number_of_clients argument will be ignored.
  -he, --he             enable homomorphic encryption.
  -i [PROJECT_INPUT], --project_input [PROJECT_INPUT]
                        project.yaml file path, If specified,
                        'number_of_clients','clients' and 'docker' specific
                        options will be ignored.
  -d [DOCKER_IMAGE], --docker_image [DOCKER_IMAGE]
                        generate docker.sh based on the docker_image, used in
            

By default, the command creates a workspace in `/tmp/nvflare/poc`. However, we can override the environment variable `NVFLARE_POC_WORKSPACE` to modify the workspace location. 

It is optional to specify a configuration file with the `-i` argument, or specifiy the number of clients: if unspecified, a default template will be used to provision startup kits for 1 server, 2 clients and 1 admin.

#### `nvflare poc prepare-jobs-dir`

This is a convenient command can be used to link any directory with federated applications to the `admin`'s `transfer` folder. As we've explained earlier, `transfer` folder of the `admin`'s startup kit can be used to hold federated applications, so that `admin` can submit an application by simplying referring to its folder name, instead of the full path. This command essentially creates a symbolic link for `admin`'s `transfer` folder to point to a directory of available federated applications.

Here is the help info for `nvflare poc prepare-jobs-dir`:

In [7]:
!nvflare poc prepare-jobs-dir -h

usage: nvflare poc prepare-jobs-dir [-h] [-j [JOBS_DIR]] [-debug]

options:
  -h, --help            show this help message and exit
  -j [JOBS_DIR], --jobs_dir [JOBS_DIR]
                        jobs directory
  -debug, --debug       debug is on


#### `nvflare poc start` and `nvflare poc stop`

As the names indicate, these command starts / stops the FL system. You can either start / stop all participants, or a specific one, for instance, starting / stopping only the `server`.

Here is the help info for `nvflare poc start` and `nvflare poc stop`:

In [8]:
!nvflare poc start -h

usage: nvflare poc start [-h] [-p [SERVICE]] [-ex [EXCLUDE]] [-gpu [GPU ...]]
                         [-debug]

options:
  -h, --help            show this help message and exit
  -p [SERVICE], --service [SERVICE]
                        participant, Default to all participants
  -ex [EXCLUDE], --exclude [EXCLUDE]
                        exclude service directory during 'start', default to ,
                        i.e. nothing to exclude
  -gpu [GPU ...], --gpu [GPU ...]
                        gpu device ids will be used as CUDA_VISIBLE_DEVICES.
                        used for poc start command
  -debug, --debug       debug is on


In [9]:
!nvflare poc stop -h

usage: nvflare poc stop [-h] [-p [SERVICE]] [-ex [EXCLUDE]] [-debug]

options:
  -h, --help            show this help message and exit
  -p [SERVICE], --service [SERVICE]
                        participant, Default to all participants
  -ex [EXCLUDE], --exclude [EXCLUDE]
                        exclude service directory during 'stop', default to ,
                        i.e. nothing to exclude
  -debug, --debug       debug is on


By default, the PoC mode starts and stops all participants at once. You can use the `-p` or `-ex` flags to include or exclude certain participants.

#### `nvflare poc clean`

This command cleans up and deletes the POC workspaces.

# Example: Starting a PoC System

Let's start a PoC system together, step by step.

> **NOTE**:
> When running the PoC mode, it's necessary to use a separate terminal since the `nvflare poc start` subcommand will run in the foreground emitting output from the server and any connected clients.
> 
> So, open a new launcher, launch a new terminal, go to this notebooks directory with `cd /flare/notebooks`, and executes all the commands in this section in that terminal.

### 1. Create a Workspace

The first step is to create a workspace for the PoC. We can use the `nvflare poc prepare` for this. 

By default, the command creates a workspace in `/tmp/nvflare/poc`. However, we can override the environment variable `NVFLARE_POC_WORKSPACE` to modify the workspace location. 

As mentioned earlier, the `prepare` subcommand calls internally the provisioning command. Without specify a configuration file or the number of clients, a default template will be used to provision startup kits for 1 server, 2 clients and 1 admin. But since we already have an example configuration file in [`../files/project.yml`/](../files/project.yml/), let go ahead and use it.

Execute the following shell commands in the opened terminal:
```shell
rm -rf $(pwd -P)/"temp-workspace/"
export NVFLARE_POC_WORKSPACE=$(pwd -P)/"temp-workspace/" && nvflare poc prepare -i ../files/project.yml
```

Running this command will create a new `temp-workspace` folder and generate an `example_project` within which we can find the startup kits for 1 server, 2 clients and 1 admin user:

In [10]:
!tree ./temp-workspace -L 3

./temp-workspace
├── data
├── example_project
│   ├── prod_00
│   │   ├── admin@nvidia.com
│   │   ├── server1
│   │   ├── site-1
│   │   └── site-2
│   ├── resources
│   │   ├── aws_template.yml
│   │   ├── azure_template.yml
│   │   └── master_template.yml
│   └── state
│       └── cert.json
└── project.yml

10 directories, 5 files


### 2. Start the PoC Mode

Next, let's use `nvflare poc start` to start the PoC mode.

Noted that `nvflare poc start` starts all participants by default, including the admin.  It's often nice to start th admin separately from the server and clients, so that we can interact with the FL system as admin later. This is exactly what we will do now. To do this, we'll pass the `-ex admin@nvidia.com` argument to exclude the admin from the PoC start-up. We will connect as admin together later.

Execute the following command in the same opened terminal:

```shell
nvflare poc start -ex admin@nvidia.com
```

After waiting for a bit, you will eventually see some console output. When you see messages similar to the following, showing that the server and clients are all up-and-running, you have successfully started a PoC system!

```
...
2024-11-14 15:40:53,558 - root - INFO - Server started
...
2024-11-14 15:40:59,995 - FederatedClient - INFO - Successfully registered client:site-1 for project example_project. Token:98da9b03-1fa8-490d-9790-a66c4148eb74 SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
...
2024-11-14 15:40:58,994 - FederatedClient - INFO - Successfully registered client:site-2 for project example_project. Token:9de44414-91ee-41a0-af2f-292a2e5477be SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
...
```


> **NOTE**:
> Please keep this terminal open for now, and do not shutdown the PoC system. We will interact with this PoC system in the next example.

# Job Submission and Project Monitoring

Now we've successfully started an FL system with server and clients in PoC mode, let's learn how to actually submit federated jobs / applications to the server, run the jobs and monitor their running status in NVIDIA FLARE.

NVIDIA FLARE offers 3 ways to submit and run federated jobs:
1. Using [`FLARE Console`](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html#operating-nvflare)
2. Using [`FLARE API`](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html)
3. Using [`FLARE Job CLI`](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/job_cli.html)

In this chapter, we focus on the first approach using `FLARE Console`. We will leave `FLARE API` as one of the exercises below for developers who want to dig deeper into the programmatic way of managing FL projects. We will not cover `FLARE Job CLI` in this course, but feel free to check out the [documentation](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/job_cli.html) to learn more about it.

The `admin` user that we just provisioned before has the right to submit jobs and manage the FL system. Therefore we need to log into the FL system as the `admin` user. This can be achieved using the `nvflare poc start` command or the start-up script of the `admin` user, as we will see in the example later. After logging into the FL system as `admin`, we will be able to manage the FL system, submit jobs and monitor the project runtime status using `FLARE Console`.

[FLARE Console](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html#operating-nvflare) is a console-like management interface for managing federated projects. FLARE Console offers various commands allowing users to manage and control the state of the FL system, including starting and stopping servers and clients, deploying applications, and monitoring deployment status. With FLARE Console, it is easy to:
- Orchestrate the entire federated learning study, allowing users to initiate and manage various components of the system.
- Deploy applications to clients and servers from the console, facilitating the execution of collaborative tasks.
- Monitor project status: the console provides real-time updates on the status of servers and clients, enabling administrators to track progress and troubleshoot issues.

FLARE Console typically runs as a standalone process on a researcher’s workstation or laptop. It interacts only with the server, not directly with FL clients. Administrators can issue commands anywhere using the console through secure network connection to manage experiments effectively.

FLARE Console offers a comprehensive list of commands to manage and monitor jobs and the overall FL system. [Here](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html#admin-command-prompt) is the list of commands available in FLARE Console. Commonly used ones include:
- `?`: use the question mark to list all available commands in FLARE Console
- `check_status`: check the status of the server or the clients
- `info`: list directories for job / application upload & download
- `submit_job`: submit a federated job / application that is visible inside the `admin`'s transfer folder
- `list_jobs`: list jobs on the server 
- `download_job`: download results of a job using the job's ID
- `shutdown`: shutdown server or specific clients
- `bye`: log out of the FLARE Console

Besides monitoring system and job runtime, another useful feature in FLARE is integration with popular [Experiment Tracking](https://nvflare.readthedocs.io/en/main/programming_guide/experiment_tracking.html) APIs and tools, for instance, Tensorboard, Weights & Biases and MLFlow. FLARE provides interfaces compatible with these experiment tracking tools, allowing developers to tracking experiment statistics in Client API during client training, and stream these statistics either locally to client workspace or to the server. We will see experiment tracking in the example later. 


Now let's put what we've learned into action: in the next section, we are going to submit a real federated application to the server, and monitor it's running status using FLARE Console!

# Example: Federated CIFAR10 Image Classification Using Pytorch

The federated application we are going submit and run is the [**Hello Pytorch**](https://github.com/NVIDIA/NVFlare/tree/main/examples/hello-world/hello-pt) example from NVIDIA FLARE's official example repository.

In this application, our goal is to train a simple neural network using Federated Averaging workflow, on the [`CIFAR10` dataset](https://www.cs.toronto.edu/~kriz/cifar.html) to perform image classification.

### Preparation

Let's copy the example to our local `exmaples` folder with the following command:

In [11]:
!if [ ! -d examples ]; then mkdir examples; fi
!if [ ! -d examples/hello-pt ]; then \
    cp -r /NVFlare/examples/hello-world/hello-pt/ examples/hello-pt; fi
!tree examples/hello-pt

examples/hello-pt
├── README.md
├── fedavg_script_runner_pt.py
├── requirements.txt
└── src
    ├── __pycache__
    │   └── simple_network.cpython-310.pyc
    ├── hello-pt_cifar10_fl.py
    └── simple_network.py

3 directories, 6 files


Let's look at this example a little bit, it should remind you want you've learned from the [previous chapter](Chapter_1_Intro_APIs_and_Simulator.ipynb) on FLARE's APIs for developing federated applications. In this Hello-Pytorch example:
- The client code is implemented in file [examples/hello-pt/src/hello-pt_cifar10_fl.py](examples/hello-pt/src/hello-pt_cifar10_fl.py). Similar to the Hello FedAvg NumPy example from previous chapter, FLARE client APIs were used to convert a straight-forward centralized Pytorch training script to federated client code, by adding just a couple of lines of API usages.
- The server code and the Job creation is in file [examples/hello-pt/fedavg_script_runner_pt.py](examples/hello-pt/fedavg_script_runner_pt.py). As a matter of fact, this example wrapped the FedAvg Controller and FedJob API calls inside a custom FedAvgJob class to make the code cleaner. But the implementation underneath is the same as the previous Hello FedAvg NumPy example.

Before moving to the next step, we need to export this example application locally as a Federated Job, so that the `admin` can submit it to the server. We've learned how to do this from the previous chapter. Let's open file [examples/hello-pt/fedavg_script_runner_pt.py](examples/hello-pt/fedavg_script_runner_pt.py), comment the last line:
```python
job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="0")
```
And uncomment the line before that:
```python
job.export_job("/tmp/nvflare/jobs/job_config")
```

Then, let's run the following command:

In [12]:
!cd examples/hello-pt/ && python3 fedavg_script_runner_pt.py

After running the command, the federated application should be exported to `/tmp/nvflare/jobs/job_config`, under the name `hello-pt_cifar10_fedavg`:

In [13]:
!tree /tmp/nvflare/jobs/job_config -L 2

/tmp/nvflare/jobs/job_config
└── hello-pt_cifar10_fedavg
    ├── app_server
    ├── app_site-1
    ├── app_site-2
    └── meta.json

5 directories, 1 file


Before starting the `admin` user, and submit the application for running, we can optionally link the location of the `hello-pt_cifar10_fedavg` application to the `admin`'s `transfer` folder. Doing so allows the `admin` to use the application's folder name for job submission. This can be achieved using the `nvflare poc prepare-jobs-dir` command. 

Let's go ahead and execute the following command:

In [14]:
!nvflare poc prepare-jobs-dir -j /tmp/nvflare/jobs/job_config

link job directory from /tmp/nvflare/jobs/job_config to /flare/notebooks/temp-workspace/example_project/prod_00/admin@nvidia.com/transfer



> **NOTE**:
> To successfully run this command, you need to make sure that the previously started PoC system is still up-and-running. To be sure of this, you can also run this command inside the terminal previously opened for starting the PoC system.

After executing this command, we can see that the `transfer` folder now points to the directory where the job `hello-pt_cifar10_fedavg` was exported:

In [15]:
!ls -l ./temp-workspace/example_project/prod_00/admin@nvidia.com/transfer

lrwxrwxrwx 1 root root 28 Nov 22 15:50 ./temp-workspace/example_project/prod_00/admin@nvidia.com/transfer -> /tmp/nvflare/jobs/job_config


### Connect to PoC System as `admin` 

With all preparation done, we are now ready to connect to our previously started PoC FL system as `admin`.

> **NOTE**:
> Please open a new terminal to execute upcoming shell commands, an interative shell session is needed.

To connect to the PoC FL system, you can either use the `nvflare poc start` subcommand, and specify only starting the `admin` user, by executing the following shell command in a new terminal:
```shell
nvflare poc start -p admin@nvidia.com
```
Or you can directly invoke the shell script `fl_admin.sh` located in the `admin` user's startup kit, by executing the following command in a new terminal:
```shell
./temp-workspace/example_project/prod_00/admin@nvidia.com/startup/fl_admin.sh
```

After connecting as the `admin` user, you will enter the [FLARE Console](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html#admin-command-prompt).

<img src="../images/flare-console.png" alt="FLARE Console" width=50% />


### Submit Job & Monitor Progress

From now on, we will stay inside the FLARE Console for all upcoming operations. 

All the following operations that we will perform can also be done using FLARE API. We will not walkthrough FLARE API now, but please feel free to refer to the [documentation](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html) to learn more. There is also an exercise on FLARE API at the end of this notebook.

To submit a job side the FLARE Console, we can use the `submit_job` command followed by either the absolute path to the federated application, or name of the application if it's been linked to admin's transfer folder. Since we've already linked the application before using `nvflare poc prepare-jobs-dir`, let's go ahead and execute the following command in the FLARE Console:

```shell
submit_job hello-pt_cifar10_fedavg
```

After a successful job submission, you should see console messages similar to the following:
```
Submitted job: bc345b01-b910-4e99-83b6-15f50d44309b
Done [65436 usecs] 2024-11-14 17:54:49.201422
```

As you can see, the job was submitted to the server for running, and a job ID of `bc345b01-b910-4e99-83b6-15f50d44309b` is assigned to keep track of the submitted job. This dynamically generated ID will change each time when you submit a job, even if it's the same job. So you will get a different one than what's shown here.

> **NOTE**:
> Remember to change the job ID in all the upcoming commands to the ID that you obtained from submitting the job.

You will also see console output messages from the terminal which you used to start the PoC system with `nvflare poc start`, indicating that the job has started running.

We can query the FL system to inspect the status of running jobs. Execute the following command in the FLARE Console:

```shell
list_jobs
```

You will get an output similar to the following:

```
--------------------------------------------------------------------------------------------------------------------------------
| JOB ID                               | NAME                    | STATUS  | SUBMIT TIME                      | RUN DURATION   |
--------------------------------------------------------------------------------------------------------------------------------
| bc345b01-b910-4e99-83b6-15f50d44309b | hello-pt_cifar10_fedavg | RUNNING | 2024-11-14T17:54:49.191296+00:00 | 0:00:03.265572 |
--------------------------------------------------------------------------------------------------------------------------------
```

Notice that the ID display with `list_jobs` corresponds to the assigned job ID after `submit_job`. We can also see that the `STATUS` of the job is `RUNNING`. The `STATUS` will become `FINISHED:COMPLETED` after the job is successfully executed. Feel free to periodically use the `list_jobs` command to check the running status of the job.

Another command to make sure that the clients are executing the job is to use `check_status client`. This might return something like this:

```
------------------------------------------------------------------------
| CLIENT | APP_NAME   | JOB_ID                               | STATUS  |
------------------------------------------------------------------------
| site-2 | app_site-2 | bc345b01-b910-4e99-83b6-15f50d44309b | started |
| site-1 | app_site-1 | bc345b01-b910-4e99-83b6-15f50d44309b | started |
------------------------------------------------------------------------
```

Indicating that both clients are executing the submitted job.

When a job is running, it will have a corresponding workspace under each participant's folder, for instance, for client `site-1`:

In [28]:
!ls -lah ./temp-workspace/example_project/prod_00/site-1/

total 308K
drwxr-xr-x 7 root root 4.0K Nov 15 20:09 .
drwxr-xr-x 6 root root 4.0K Nov 14 16:51 ..
drwxr-xr-x 2 root root 4.0K Nov 14 19:57 .ipynb_checkpoints
-rw-r--r-- 1 root root 9.0K Nov 15 19:55 audit.log
drwxr-xr-x 3 root root 4.0K Nov 15 18:05 bc345b01-b910-4e99-83b6-15f50d44309b
-rw-r--r-- 1 root root 246K Nov 15 19:55 cifar_net.pth
-rw-r--r-- 1 root root    4 Nov 14 16:52 daemon_pid.fl
drwxr-xr-x 2 root root 4.0K Nov 14 16:51 local
-rw-r--r-- 1 root root 5.3K Nov 15 19:55 log.txt
-rw-r--r-- 1 root root    4 Nov 14 16:52 pid.fl
-rw-r--r-- 1 root root  746 Nov 14 16:51 readme.txt
drwxr-xr-x 2 root root 4.0K Nov 14 16:51 startup
drwxr-xr-x 2 root root 4.0K Nov 14 16:51 transfer


You will see a folder having it's name same as the job ID (`bc345b01-b910-4e99-83b6-15f50d44309b` in this case), that represents the job's workspace on client `site-1`. You will find the same workspace on `server` and `site-2` as well when the job is still running.

After the job is finishes, the server side job workspace will be moved to a secure filesystem location by default. There are ways to configure how and where server-side job workspace are stored after a job finishes, but we will not go into details of this during this course. To be able to retrieve the server's workspace, which contains the final global model and important logs, we need to issue the `download_job` command:

```shell
download_job bc345b01-b910-4e99-83b6-15f50d44309b
```

Go ahead and execute this command in the FLARE Console and remember to replace the job ID with yours.

After the job downloads successfully, you should see message similar to the following:
```
> download_job bc345b01-b910-4e99-83b6-15f50d44309b
downloading job.zip ...
downloaded job.zip (45532 bytes) in 0.015504598617553711 seconds
downloading meta.json ...
downloaded meta.json (792 bytes) in 0.01458883285522461 seconds
downloading workspace.zip ...
downloaded workspace.zip (315405 bytes) in 0.01614093780517578 seconds
content downloaded to /flare/notebooks/temp-workspace/example_project/prod_00/admin@nvidia.com/startup/../transfer/bc345b01-b910-4e99-83b6-15f50d44309b
Done [67753 usecs] 2024-11-15 20:18:18.540653
```

This command will download the server-side job workspace (whole content of folder `bc345b01-b910-4e99-83b6-15f50d44309b`) to the `transfer` folder of the `admin` user:

In [30]:
!tree ./temp-workspace/example_project/prod_00/admin@nvidia.com/transfer/bc345b01-b910-4e99-83b6-15f50d44309b -L 3

./temp-workspace/example_project/prod_00/admin@nvidia.com/transfer/bc345b01-b910-4e99-83b6-15f50d44309b
├── job
│   └── hello-pt_cifar10_fedavg_extproc
│       ├── app_server
│       ├── app_site-1
│       ├── app_site-2
│       └── meta.json
├── meta.json
└── workspace
    ├── app_server
    │   ├── FL_global_model.pt
    │   ├── config
    │   └── custom
    ├── cross_site_val
    │   └── cross_val_results.json
    ├── fl_app.txt
    ├── log.txt
    ├── meta.json
    ├── stats_pool_summary.json
    └── tb_events
        ├── site-1
        └── site-2

14 directories, 8 files


You can see that the server side workspace contains the application itself, the final global model `FL_global_model.pt`, as well as many other log files. Feel free to checkout also client side workspaces: they still remain at client folders after a job finishes.


As we've mentioned earlier, FLARE offers the capability to track experiments with popular tools such as Tensorboard, Weights & Biases and MLFlow. This Hello Pytorch example application actually uses the [`SummaryWriter`](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/client/tracking.py#L23) API inside client code (line 79 of [examples/hello-pt/src/hello-pt_cifar10_fl.py](examples/hello-pt/src/hello-pt_cifar10_fl.py)) to allow metrics logging in the same way as Tensorboard. 

During client training, these metrics are streamed to the server's workspace as Tensorboard-readable event format. As we've mentioned earlier, if the job is still running, the server side workspace will be inside the server folder. Therefore we can monitor real-time metrics of the training process by launching the `tensorboard` command, and point it to the server folder. 

After the job finishes and after we've downloaded the server side job workspace to the `admin`'s transfer folder, we can inspect the training metrics using the following tensorboard commands:

In [16]:
%load_ext tensorboard

In [29]:
%tensorboard --logdir ./temp-workspace/example_project/prod_00/admin@nvidia.com/transfer/bc345b01-b910-4e99-83b6-15f50d44309b/workspace/tb_events/ --bind_all

This example application did not really calculate meaningfull validation metrics during the training process. But you can imagine that within your own real-world application, you could easily log any client side metrics such as accuracy, precision, recall etc., and monitor them in real-time as the training progresses!


**That's it, we have successfully provisioned a PoC system and deployed a real federated application!**

Last but not least, do not forget the shut the PoC system down using the following command:

In [34]:
!nvflare poc stop

start shutdown NVFLARE
connect to nvflare server
checking running jobs
shutdown NVFLARE
waiting system to shutdown


When you are ready, use the following command to clean up and delete the workspace generated by the PoC system:

In [ ]:
!nvflare poc clean

# Recap

Let us recap what we've learned in this chapter:
- We first learned about FLARE's provisioning process, a crucial part for real-world deployment of federated projects.
- We then had an introduction to FLARE's PoC mode, a runtime environment allowing developers to provision an FL system in a sandbox environment on a single PC. We also started a PoC system together. 
- Next, we walked through the FLARE Console and saw how it can be used to submit federated applications to the FL system for running, manage and monitor job status. We also learned about FLARE's integration with experiment tracking tools such as Tensorboard, Weights & Biases, MLFlow.
- Finally, we deployed a real federated application with FLARE Console for CIFAR10 image classification using Pytorch, performed project monitoring with FLARE Console and experiment tracking with Tensorboard.

With the knowledge of chapter 2 and chapter 3, you are now able to develop a federated application using FLARE, provision a deployment system and deploy the federated application! There is still some knowledge to gain, if you want your federated application to be fully bullet-proof and scalable in a real-world environment. In the next chapter, we will introduce some of FLARE's advanced features that will hopefully allow you to grasp the last bit of skills towards a full-fledged federated learning developer!

Before continuing to the [next chapter](Chapter_4_Advanced_Topics_Use_Cases_and_Additional_Resources.ipynb), here are some exercises to help you grasp the essential of this chapter!

# Exercise 1

Modify the project configuration file [../files/project.yml](../files/project.yml) to add another client `site-3`. Provision a PoC system with the new modified configuration file. Then start up the PoC system with only the server (`server1`) and the new client (`site-3`). What would happen if you start `site-3` before the `server`?

## Solution to Exercise 1

Modify [../files/project.yml](../files/project.yml) by adding the following lines at anywhere is the `participants` section (for exmaple, at the end):
```yaml
  - name: site-3
    type: client
    org: nvidia
```

Provision the PoC system using `nvflare poc prepare`:
```shell
rm -rf $(pwd -P)/"temp-workspace/"
export NVFLARE_POC_WORKSPACE=$(pwd -P)/"temp-workspace/" && nvflare poc prepare -i ../files/project.yml
```

Start the PoC system with `nvflare poc start`. Using flag `-p` to start first `server1`, then `site-3`:
```shell
nvflare poc start -p server1 
nvflare poc start -p site-3
```

If you start the client before the server, the start-up will fail since there is no server endpoint for the client to connect to.

# Exercise 2

Restart the PoC system. This time, use FLARE API to establish an admin user connection. After that, submit the same Hello Pytorch application, query job status and download the job when it finishes, all using FLARE API. For references, you can find documentations for FLARE API [here](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html), [here](https://nvflare.readthedocs.io/en/main/real_world_fl/migrating_to_flare_api.html) and [here](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.fuel.flare_api.flare_api.html#module-nvflare.fuel.flare_api.flare_api).

## Solution to Exercise 2

First, prepare and start up the PoC system (only server and clients) by running the following commands in a new terminal:

> **Note**:
> Don't forget to remove the section on `site-3` if you have previously modified the project.yml configuration file during Exercise 1.

```shell
rm -rf $(pwd -P)/"temp-workspace/"
export NVFLARE_POC_WORKSPACE=$(pwd -P)/"temp-workspace/" && nvflare poc prepare -i files/project.yml

nvflare poc start -ex admin@nvidia.com
```

To connect to the PoC system as admin user, use the following FLARE APIs:

In [26]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session

project_name = "example_project"
username = "admin@nvidia.com"
admin_user_dir = os.path.join(os.getcwd(), "temp-workspace", project_name, "prod_00", username)

sess = new_secure_session(
    username=username,
    startup_kit_location=admin_user_dir
)
print(sess.get_system_info())

SystemInfo
server_info:
status: stopped, start_time: Sat Nov 23 00:37:44 2024
client_info:
site-2(last_connect_time: Sat Nov 23 00:38:11 2024)
site-1(last_connect_time: Sat Nov 23 00:38:12 2024)
job_info:



To submit the `hello-pt_cifar10_fedavg` application, use the following API. Noted here we submit the application using its full path, instead of linking the application's folder to `admin`'s `transfer` folder first.

In [39]:
path_to_example_job = "/tmp/nvflare/jobs/job_config/hello-pt_cifar10_fedavg"
job_id = sess.submit_job(path_to_example_job)
print(job_id + " was submitted")


07d7607c-4b1a-471d-a52f-e21b9c953cad was submitted


Note down the job ID, and use the following API to monitor the live job status:

In [40]:
from nvflare.fuel.flare_api.flare_api import basic_cb_with_print

job_id = "07d7607c-4b1a-471d-a52f-e21b9c953cad"
sess.monitor_job(job_id, cb=basic_cb_with_print, cb_run_counter={"count":0})

{'name': 'hello-pt_cifar10_fedavg', 'resource_spec': {}, 'min_clients': 1, 'deploy_map': {'app_server': ['server'], 'app_site-1': ['site-1'], 'app_site-2': ['site-2']}, 'byoc': True, 'submitter_name': 'admin@nvidia.com', 'submitter_org': 'nvidia', 'submitter_role': 'project_admin', 'job_folder_name': 'hello-pt_cifar10_fedavg', 'job_id': '07d7607c-4b1a-471d-a52f-e21b9c953cad', 'submit_time': 1731709802.0272186, 'submit_time_iso': '2024-11-15T22:30:02.027219+00:00', 'start_time': '2024-11-15 22:30:02.746792', 'duration': 'N/A', 'data_storage_format': 2, 'status': 'RUNNING', 'job_deploy_detail': ['server: OK', 'site-1: OK', 'site-2: OK'], 'schedule_count': 1, 'last_schedule_time': 1731709802.698105, 'schedule_history': ['2024-11-15 22:30:02: scheduled']}
{'name': 'hello-pt_cifar10_fedavg', 'resource_spec': {}, 'min_clients': 1, 'deploy_map': {'app_server': ['server'], 'app_site-1': ['site-1'], 'app_site-2': ['site-2']}, 'byoc': True, 'submitter_name': 'admin@nvidia.com', 'submitter_org': 

<MonitorReturnCode.JOB_FINISHED: 0>

The `cb` argument can be used to provide your own custom callback function for status monitoring.

You can also perform `list_jobs` with the following API:

In [42]:
sess.list_jobs(detailed=True)

[{'name': 'hello-pt_cifar10_fedavg',
  'resource_spec': {},
  'min_clients': 1,
  'deploy_map': {'app_server': ['server'],
   'app_site-1': ['site-1'],
   'app_site-2': ['site-2']},
  'byoc': True,
  'submitter_name': 'admin@nvidia.com',
  'submitter_org': 'nvidia',
  'submitter_role': 'project_admin',
  'job_folder_name': 'hello-pt_cifar10_fedavg',
  'job_id': '07d7607c-4b1a-471d-a52f-e21b9c953cad',
  'submit_time': 1731709802.0272186,
  'submit_time_iso': '2024-11-15T22:30:02.027219+00:00',
  'start_time': '2024-11-15 22:30:02.746792',
  'duration': '0:02:45.010479',
  'data_storage_format': 2,
  'status': 'FINISHED:COMPLETED',
  'job_deploy_detail': ['server: OK', 'site-1: OK', 'site-2: OK'],
  'schedule_count': 1,
  'last_schedule_time': 1731709802.698105,
  'schedule_history': ['2024-11-15 22:30:02: scheduled']}]

When the job finishes, download the server-side job workspace to the `admin`'s `transfer` folder using the following API:

In [43]:
job_download = sess.download_job_result(job_id)
print("Job download path: " + job_download)

Job download path: /flare/notebooks/temp-workspace/example_project/prod_00/admin@nvidia.com/transfer/07d7607c-4b1a-471d-a52f-e21b9c953cad


In [46]:
!tree {job_download} -L 3

/flare/notebooks/temp-workspace/example_project/prod_00/admin@nvidia.com/transfer/07d7607c-4b1a-471d-a52f-e21b9c953cad
├── job
│   └── hello-pt_cifar10_fedavg
│       ├── app_server
│       ├── app_site-1
│       ├── app_site-2
│       └── meta.json
├── meta.json
└── workspace
    ├── app_server
    │   ├── FL_global_model.pt
    │   ├── config
    │   └── custom
    ├── cross_site_val
    │   └── cross_val_results.json
    ├── fl_app.txt
    ├── log.txt
    ├── meta.json
    ├── stats_pool_summary.json
    └── tb_events
        ├── site-1
        └── site-2

14 directories, 8 files


Finally, shutdown the PoC system and close the admin connection:

In [ ]:
sess.shutdown("client")
sess.shutdown("server")

# Or
# print(sess.api.do_command("shutdown client"))
# print(sess.api.do_command("shutdown server"))

sess.close()